In [1]:
import os
import datetime
import logging
import shutil
import pandas as pd
import glob
import csv

from FUNCTION import make_top_protein, fill_water_ions, energy_min, make_new_minim_nvt_npt, make_new_minim_config_samd, run_md
from FUNCTION import files_gmxmmpbsa, gmx_mmpbsa, Data_Analysis_Pre, Data_Analysis_Cal, clean_for_each_cycle

#PROJECT_ROOT = os.path.dirname(os.path.abspath(__file__))
PROJECT_ROOT = os.getcwd()

DATA_DIR = os.path.join(PROJECT_ROOT, "DATA")
VMD_DIR = os.path.join(PROJECT_ROOT, "VMD_FUNCTION")
FUNCTION_DIR = os.path.join(PROJECT_ROOT, "FUNCTION")
FORCE_FIELD_PATH = os.path.join(PROJECT_ROOT, "FORCE_FIELD")
MMPBSA_INFILE_PATH = os.path.join(PROJECT_ROOT, "gmx_mmpbsa_in")
# pdb file
protein_infile = "HLA_BiAB_protein_50ns" 
#protein_infile = "mtbind"
protein_file_path = os.path.join(DATA_DIR, f"{protein_infile}.pdb")

# MDP files
ions_mdp_file = "ions"
minim_mdp_file = "minim"
nvt_mdp_file = "NVT"
npt_mdp_file = "NPT"
samd_mdp_file = "SAMD"
md_mdp_file = "EngComp_ff14sb_custom"
only_protein_md_mdp_file = "Protein_EngComp_ff14sb_custom"

ions_mdp_path = os.path.join(DATA_DIR, f"{ions_mdp_file}.mdp")
minim_mdp_path = os.path.join(DATA_DIR, f"{minim_mdp_file}.mdp")
nvt_mdp_path = os.path.join(DATA_DIR, f"{nvt_mdp_file}.mdp")
npt_mdp_path = os.path.join(DATA_DIR, f"{npt_mdp_file}.mdp")
samd_mdp_path = os.path.join(DATA_DIR, f"{samd_mdp_file}.mdp")
md_mdp_path = os.path.join(DATA_DIR, f"{md_mdp_file}.mdp")
only_protein_md_mdp_path = os.path.join(DATA_DIR, f"{only_protein_md_mdp_file}.mdp")



def create_output_directory():
    
    current_dir = os.getcwd()
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    output_dir_name = f"output_{timestamp}"
    output_dir_path = os.path.join(current_dir, output_dir_name)
    
    os.mkdir(output_dir_path)
    print(f"Created directory: {output_dir_path}")

    os.chdir(output_dir_path)

    
    
    return output_dir_path

In [2]:
ROOT_OUTPUT = create_output_directory()

logging.basicConfig(
    filename = "OUTPUT.out",
    level = logging.INFO,
    format="%(asctime)s - %(levelname)s -%(message)s"
)

logging.info(f"PATH: {ROOT_OUTPUT}")

Created directory: /home/bio/Desktop/jupyter_test/antibody_test/output_20241203_175732


In [3]:
# create configuration folder
configuration_path = os.path.join(os.getcwd(),"configuration")
os.mkdir(configuration_path)
print(f"Create directory: {configuration_path}")
os.chdir(configuration_path)

Create directory: /home/bio/Desktop/jupyter_test/antibody_test/output_20241203_175732/configuration


In [4]:
# build folders
# cycle_num, MAKE IT 1 NOW, NEED TO CHANGE
cycle_num = 2
def build_folders(current_dir, cycle_num):
    # Create folder for each cycle
    folders  ={}
    
    for cycle_n in range (1,cycle_num + 1):
        folder_name = f"cycle{cycle_n}_MD"
        folder_path = os.path.join(current_dir, folder_name)
        os.makedirs(folder_path, exist_ok = True)
        folders[f"cycle{cycle_n}_MD"] = folder_path

    folders["repository"] = os.path.join(current_dir,"REPOSITORY")
    folders["TEMP_FILES_FOLDER"] = os.path.join(current_dir,"TEMP_FILES_FOLDER")
    folders["REMOVED_FILES_FOLDER"] = os.path.join(current_dir,"REMOVED_FILES_FOLDER")
    folders["results"] = os.path.join(current_dir,"RESULTS")

    for folder in folders.values():
        os.makedirs(folder,exist_ok = True)
  
    header = [
    "#RUNnumber", "DeltaG(kJ/mol)", "Coul(kJ/mol)", "vdW(kJ/mol)",
    "PolSol(kJ/mol)", "NpoSol(kJ/mol)", "ScoreFunct", "ScoreFunct2",
    "Canonica_AVG", "MedianDG", "DeltaG_2s", "dG_PotEn"]

    df = pd.DataFrame(columns=header)
    results_file_path = os.path.join(folders["results"], "MoleculesResults.dat")
    df.to_csv(results_file_path, sep='\t', index=False, header=True)

    return folders
    

In [5]:
current_dir = os.getcwd()
folders = build_folders(current_dir,cycle_num)

In [6]:
# generating a topology and build box
make_top_protein(protein_file_path, "amber99sb-ildn", "tip3p", "system", "topol")

In [7]:
# cp system.pdb {protein_infile}.pdb in current folder
source = os.path.join(current_dir, "system.pdb")
destination = os.path.join(current_dir, f"{protein_infile }.pdb")
try:
    shutil.copy(source,destination)
except Exception:
    print("Copy system.pdb failed.")

In [8]:
## OC2 DOESN'T NORMAL ONE, NEED TO CHANGE
def add_ter_to_pdb(pdb_file_name):
    
    temp_file_name = f"{pdb_file_name}_temp"  
    with open(pdb_file_name, 'r') as f:
        lines = f.readlines()

    new_lines = []  
    for i, line in enumerate(lines):
        new_lines.append(line)
        if "OC2" in line:
            if i + 1 >= len(lines) or not lines[i + 1].startswith("TER"):
                new_lines.append("TER\n")

    with open(temp_file_name, 'w') as f:
        f.writelines(new_lines)

    os.rename(temp_file_name, pdb_file_name)

In [9]:
add_ter_to_pdb(f"{protein_infile }.pdb")

In [10]:
def replace_his_(input_pdb, output_pdb):
    # input_pdb: the pdb in configuration/mutant x
    # output_pdb: make it in ROOT_OUTPUT folder
    with open(input_pdb, 'r') as infile:
        data = infile.read()
    data = data.replace("HISD", "HIS").replace("HISE", "HIS").replace("HISP", "HIS")

    with open(output_pdb, 'w') as outfile:
        outfile.write(data)

In [11]:
output_pdb = os.path.join(ROOT_OUTPUT, f"{protein_infile}.pdb")
replace_his_(f"{protein_infile}.pdb",output_pdb)

In [12]:
# Adding water and ions
fill_water_ions("system", "topol", ions_mdp_path)
# Energy Minimiization
energy_min(minim_mdp_path, "system_ions", "topol", "system_compl")

In [13]:
# RESIDUAL SELECTION

In [13]:
# Nvt and Npt
sequence = 0
make_new_minim_nvt_npt("system_compl_minim.gro", nvt_mdp_path, npt_mdp_path, "system_equil", 0)


Statistics over 50001 steps [ 0.0000 through 50.0000 ps ], 1 data sets
All statistics are over 501 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 293.922        6.4    18.9871    34.6693  (K)

Statistics over 50001 steps [ 0.0000 through 100.0000 ps ], 2 data sets
All statistics are over 501 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Pressure                   -2.66211         --    95.8876    13.2411  (bar)
Density                     1010.66         --    1.17675   0.658474  (kg/m^3)


                      :-) GROMACS - gmx energy, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/antibody_test/output_20241203_175732/configuration
Command line:
  gmx energy -f NVT.edr -o temp_NVT.xvg

Opened NVT.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Position-Rest.  12  Potential     
 13  Kinetic-En.     14  Total-Energy    15  Conserved-En.   16  Temperature   
 17  Pres.-DC        18  Pressure        19  Constr.-rmsd    20  Vir-XX        
 21  Vir-XY          22  Vi

In [14]:
# Move .cpt, .top, and .itp files to repository folder
for file_pattern in [f"{current_dir}/*.cpt", f"{current_dir}/*.top", f"{current_dir}/*.itp"]:
    for file in glob.glob(file_pattern):
        shutil.move(file, folders["repository"])

# Move specific files to repository folder
shutil.move(f"{current_dir}/{protein_infile}.pdb", folders["repository"])
shutil.move(f"{current_dir}/system_compl_minim.gro", folders["repository"])
shutil.move(f"{current_dir}/system_equil.gro", folders["repository"])


# Move temp* and *out files to removed files folder
for file in glob.glob("./*temp*.*") + glob.glob("./*.temp") + glob.glob("./*out"):
    shutil.move(file, folders["REMOVED_FILES_FOLDER"])

# Remove files with # in their name
for file in glob.glob("./#*"):
    os.remove(file)


In [15]:
def MD_for_each_cycle(cycle_num,input_structure_file,samd_mdp_path,output_gro, sequence, md_mdp_path, tpr_file, trj_name):
    cycle_number = 1
    while cycle_number <= cycle_num:
        
        cycle_MD_path = folders[f"cycle{cycle_number}_MD"] 
        os.chdir(cycle_MD_path)
        shutil.copy(os.path.join(folders["repository"], "system_equil.gro"), "./")
        shutil.copy(os.path.join(folders["repository"], "topol.top"), "./")

        for itp_file in glob.glob(os.path.join(folders["repository"], "*rotein_chain_*.itp")):
            shutil.copy(itp_file, "./")

        for itp_file in glob.glob(os.path.join(folders["repository"], "posres_*.itp")):
            shutil.copy(itp_file, "./")

        for cpt_file in glob.glob(os.path.join(folders["repository"], "*NPT*.cpt")):
            shutil.copy(cpt_file, "./")

        #make_new_minim_config_samd("system_equil.gro", samd_mdp_path, "system_Compl_MDstart", 0)
        make_new_minim_config_samd(input_structure_file, samd_mdp_path, output_gro, sequence)
        #run_md(md_mdp_path,"system_Compl_MD", "traj_MD", 0, 1)
        run_md(md_mdp_path, tpr_file, trj_name, sequence, cycle_number)
        shutil.copy("system_Compl_MD.gro", f"LastFrame_cycle{cycle_number}.gro")
        cycle_number += 1

In [16]:
cycle_num =2
sequence = 0
MD_for_each_cycle(cycle_num,"system_equil.gro",samd_mdp_path,"system_Compl_MDstart", sequence, md_mdp_path, "system_Compl_MD", "traj_MD")

                      :-) GROMACS - gmx energy, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/antibody_test/output_20241203_175732/configuration/cycle1_MD
Command line:
  gmx energy -f SAMD.edr -o press_SAMD.xvg

Opened SAMD.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Potential       12  Kinetic-En.   
 13  Total-Energy    14  Conserved-En.   15  Temperature     16  Pres.-DC      
 17  Pressure        18  Constr.-rmsd    19  Box-X           20  Box-Y         
 21  Box-Z   


Statistics over 175001 steps [ 0.0000 through 350.0000 ps ], 3 data sets
All statistics are over 1751 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 339.901         13    27.1075   -51.2275  (K)
Pressure                    1.12466         11    96.9937   -46.9698  (bar)
Density                     981.336         13    27.8205     47.024  (kg/m^3)
Selected 1: 'Protein'
Selected 1: 'Protein'
SAMD completed successfully!
18:00:15 -- Running MD 


                      :-) GROMACS - gmx energy, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/antibody_test/output_20241203_175732/configuration/cycle1_MD
Command line:
  gmx energy -f PROD.edr -o PROD0.xvg

Opened PROD.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Potential       12  Kinetic-En.   
 13  Total-Energy    14  Conserved-En.   15  Temperature     16  Pres.-DC      
 17  Pressure        18  Constr.-rmsd    19  Box-X           20  Box-Y         
 21  Box-Z        


Statistics over 2500001 steps [ 0.0000 through 5000.0000 ps ], 3 data sets
All statistics are over 25001 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 309.968      0.012    1.21522  0.0312671  (K)
Pressure                    2.37375        1.1    99.2421     1.4588  (bar)
Density                     1011.93      0.062    2.02794   0.251669  (kg/m^3)


Reading frame     200 time 4000.000   

GROMACS reminds you: "Step On the Brakes" (2 Unlimited)



Selected 1: 'Protein'
Selected 1: 'Protein'


                      :-) GROMACS - gmx energy, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/antibody_test/output_20241203_175732/configuration/cycle2_MD
Command line:
  gmx energy -f SAMD.edr -o press_SAMD.xvg

Opened SAMD.edr as single precision energy file

Select the terms you want from the following list by
selecting either (part of) the name or the number or a combination.
End your selection with an empty line or a zero.
-------------------------------------------------------------------
  1  Bond             2  Angle            3  Proper-Dih.      4  Per.-Imp.-Dih.
  5  LJ-14            6  Coulomb-14       7  LJ-(SR)          8  Disper.-corr. 
  9  Coulomb-(SR)    10  Coul.-recip.    11  Potential       12  Kinetic-En.   
 13  Total-Energy    14  Conserved-En.   15  Temperature     16  Pres.-DC      
 17  Pressure        18  Constr.-rmsd    19  Box-X           20  Box-Y         
 21  Box-Z   


Statistics over 175001 steps [ 0.0000 through 350.0000 ps ], 3 data sets
All statistics are over 1751 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 340.001         13    27.1019   -51.0645  (K)
Pressure                   0.722892         11    96.1507   -49.0189  (bar)
Density                     981.445         13     28.275    47.7707  (kg/m^3)
Selected 1: 'Protein'
Selected 1: 'Protein'
SAMD completed successfully!
18:11:36 -- Running MD 


In [17]:
def gmx_mmpbsa_for_each_cycle(cycle_num,only_protein_md_mdp_path,VMD_DIR,temp_files_folder, FORCE_FIELD_PATH, MMPBSA_INFILE_PATH, REMOVED_FILES_FOLDER, results_folder, repository_folder, current_conf_path):
    cycle_number = 1
    while cycle_number <= cycle_num:
        ConfName = f"cycle{cycle_number}"
        RootName = f"cycle{cycle_number}_BE"
        cycle_number_MD_FOLDER = folders[f"cycle{cycle_number}_MD"]
        # 输出相关信息
        print(f"Cycle Number: {cycle_number}")
        print(f"Configuration Name: {ConfName}")
        print(f"Root Name: {RootName}")
        print(f"MD Folder Path: {cycle_number_MD_FOLDER}")
        os.chdir(cycle_number_MD_FOLDER )
        
        repository_pdb_file = os.path.join(repository_folder, f"{protein_infile}.pdb")
        #startingFrameGMXPBSA="2000"
        # make files for gmx_mmpbsa
        # files_gmxmmpbsa(starting_gro_file, repository_pdb_file, trj_file, tpr_file, top_file, mdp_name, root_name, conf_name, vmd_function_folder, temp_files_folder)

        files_gmxmmpbsa("system_Compl_MD", repository_pdb_file, "traj_MD", "system_Compl_MD", "topol", only_protein_md_mdp_path, RootName, ConfName, VMD_DIR, temp_files_folder, cycle_number)
        # get number of frames
        try:
            with open("trj_check.out", "r") as file:
                number_of_frames = next(
                    (line.split()[1] for line in file if line.startswith("Step")), None
                )
        except FileNotFoundError:
            print(f"Error: File trj_check.out not found.")
            number_of_frames = None
        #conda_activate_path="/home/bio/ls/bin"
        conda_path = shutil.which("conda")
        conda_activate_path = os.path.dirname(conda_path)
        conda_gmxmmpbsa_name="gmxMMPBSA"
        forcefield="amber99sb-ildn"
        #FORCE_FIELD_PATH = "/home/bio/Desktop/jupyter_test/antibody_test/FORCE_FIELD"
        mmpbsa_inFILE="mmpbsa_LinearPB_amber99SB_ILDN.in"
        #MMPBSA_INFILE_PATH = "/home/bio/Desktop/jupyter_test/antibody_test/gmx_mmpbsa_in"
        np_value = 32
        #gmx_mmpbsa(1, conda_activate_path, conda_gmxmmpbsa_name, cycle_number_MD_FOLDER, ConfName, RootName, forcefield, FORCE_FIELD_PATH, 
        #             mmpbsa_inFILE, MMPBSA_INFILE_PATH , np_value, number_of_frames)
        gmx_mmpbsa(cycle_number, conda_activate_path, conda_gmxmmpbsa_name, cycle_number_MD_FOLDER, ConfName, RootName, forcefield, FORCE_FIELD_PATH, mmpbsa_inFILE, MMPBSA_INFILE_PATH, np_value, number_of_frames)
        # data analysis
        NUMframe = "all"
        Data_Analysis_Pre(cycle_number_MD_FOLDER, REMOVED_FILES_FOLDER, NUMframe)
        Data_Analysis_Cal(cycle_number, results_folder)
        # clean and move files
        clean_for_each_cycle(cycle_number, repository_folder, cycle_number_MD_FOLDER, RootName, REMOVED_FILES_FOLDER, current_conf_path)
            # 更新周期号及相关变量
        cycle_number += 1
        #conf_name = f"cycle{cycle_number}"
        #root_name = f"cycle{cycle_number}_BE"
        #cycle_number_md_folder = os.path.join(current_conf_path, f"cycle{cycle_number}_MD")
    
    

In [18]:
gmx_mmpbsa_for_each_cycle(cycle_num, only_protein_md_mdp_path,VMD_DIR,folders["TEMP_FILES_FOLDER"], FORCE_FIELD_PATH, MMPBSA_INFILE_PATH, folders["REMOVED_FILES_FOLDER"], folders["results"], folders["repository"], configuration_path)


Cycle Number: 1
Configuration Name: cycle1
Root Name: cycle1_BE
MD Folder Path: /home/bio/Desktop/jupyter_test/antibody_test/output_20241203_175732/configuration/cycle1_MD
		--running MAKE_NDX to create index.ndx..
Going to read 0 old index file(s)
Analysing residue names:
There are:   413    Protein residues
There are: 23061      Water residues
There are:   141        Ion residues
Analysing Protein...

  0 System              : 75722 atoms
  1 Protein             :  6398 atoms
  2 Protein-H           :  3292 atoms
  3 C-alpha             :   413 atoms
  4 Backbone            :  1239 atoms
  5 MainChain           :  1656 atoms
  6 MainChain+Cb        :  2035 atoms
  7 MainChain+H         :  2063 atoms
  8 SideChain           :  4335 atoms
  9 SideChain-H         :  1636 atoms
 10 Prot-Masses         :  6398 atoms
 11 non-Protein         : 69324 atoms
 12 Water               : 69183 atoms
 13 SOL                 : 69183 atoms
 14 non-Water           :  6539 atoms
 15 Ion                

                     :-) GROMACS - gmx make_ndx, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/antibody_test/output_20241203_175732/configuration/cycle1_MD
Command line:
  gmx make_ndx -f system_Compl_MD.gro -o index.ndx


Reading structure file

GROMACS reminds you: "Somewhere, something incredible is waiting to be known." (Carl Sagan)



GRO to PDB completed successfully: HLA_BiAB_protein_50ns
		--running TRJCONV to remove the pbc from the trajectory..
		--TRJCONV completed successfully!
		--running MAKE_NDX to make index with only receptor, ligand and complex..
		--MAKE_NDX completed successfully!
		--Creating a protein-only topology file...
		--Protein topology file created successfully!
		--running GROMPP to make a protein tpr..
		--GROMPP completed successfully!
		--Counting HIS residues in the PDB file..
		--Found HIS residues: 0
Files created successfully!
NP_value=32 	 number_of_frames=151 	 NP_used=16
Finished gmx_MMPBSA on cycle1
Delta Energy Terms extracted and saved to delta_energy_terms.csv
Cycle Number: 2
Configuration Name: cycle2
Root Name: cycle2_BE
MD Folder Path: /home/bio/Desktop/jupyter_test/antibody_test/output_20241203_175732/configuration/cycle2_MD
		--running MAKE_NDX to create index.ndx..
Going to read 0 old index file(s)
Analysing residue names:
There are:   413    Protein residues
There are: 

/home/bio/Desktop/jupyter_test/antibody_test/FUNCTION/Data_Analysis.py:180: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(input_file, delim_whitespace=True, header=None)
/home/bio/Desktop/jupyter_test/antibody_test/FUNCTION/Data_Analysis.py:203: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  DeltaG_temp = row[1]  # 使用列名访问 DeltaG
                     :-) GROMACS - gmx make_ndx, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/antibody_test/output_20241203_175732/configuration/cycle2_MD
Command line:
  gmx make_ndx -f system_Compl_MD.gro -o index.ndx


Reading structure file

GROMACS reminds you: "Shit Happen

GRO to PDB completed successfully: HLA_BiAB_protein_50ns
		--running TRJCONV to remove the pbc from the trajectory..
Error executing command: ['bash', '-c', 'gmx check -f ./cycle1_BE/cycle1_noPBC.xtc > trj_check.out 2>&1']
Command failed: 


RuntimeError: Command failed: 

In [19]:
#ConfName = f"cycle{cycle_number}"
ConfName = "cycle1"
#RootName = f"cycle{cycle_number}_BE"
RootName = "cycle1_BE"
cycle_number_MD_FOLDER = folders["cycle1_MD"]

In [20]:
os.chdir(cycle_number_MD_FOLDER )

In [21]:
repository_pdb_file = os.path.join(folders["repository"], f"{protein_infile}.pdb")
#startingFrameGMXPBSA="2000"
# make files for gmx_mmpbsa
# files_gmxmmpbsa(starting_gro_file, repository_pdb_file, trj_file, tpr_file, top_file, mdp_name, root_name, conf_name, vmd_function_folder, temp_files_folder)

files_gmxmmpbsa("system_Compl_MD", repository_pdb_file, "traj_MD", "system_Compl_MD", "topol", only_protein_md_mdp_path, RootName, ConfName, VMD_DIR, folders["TEMP_FILES_FOLDER"])


		--running MAKE_NDX to create index.ndx..
Going to read 0 old index file(s)
Analysing residue names:
There are:   413    Protein residues
There are: 23061      Water residues
There are:   141        Ion residues
Analysing Protein...

  0 System              : 75722 atoms
  1 Protein             :  6398 atoms
  2 Protein-H           :  3292 atoms
  3 C-alpha             :   413 atoms
  4 Backbone            :  1239 atoms
  5 MainChain           :  1656 atoms
  6 MainChain+Cb        :  2035 atoms
  7 MainChain+H         :  2063 atoms
  8 SideChain           :  4335 atoms
  9 SideChain-H         :  1636 atoms
 10 Prot-Masses         :  6398 atoms
 11 non-Protein         : 69324 atoms
 12 Water               : 69183 atoms
 13 SOL                 : 69183 atoms
 14 non-Water           :  6539 atoms
 15 Ion                 :   141 atoms
 16 Water_and_ions      : 69324 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom       '&': and  'del' nr    

                     :-) GROMACS - gmx make_ndx, 2024.4 (-:

Executable:   /opt/gromacs-2024.4/bin/gmx
Data prefix:  /opt/gromacs-2024.4
Working dir:  /home/bio/Desktop/jupyter_test/antibody_test/output_20241203_123324/configuration/cycle1_MD
Command line:
  gmx make_ndx -f system_Compl_MD.gro -o index.ndx


Reading structure file

GROMACS reminds you: "Our struggle today is not to have a female Einstein get appointed as an assistant professor. It is for a woman schlemiel to get as quickly promoted as a male schlemiel." (Bella Abzug)



GRO to PDB completed successfully: HLA_BiAB_protein_50ns
		--running TRJCONV to remove the pbc from the trajectory..
		--TRJCONV completed successfully!
		--running MAKE_NDX to make index with only receptor, ligand and complex..
		--MAKE_NDX completed successfully!
		--Creating a protein-only topology file...
		--Protein topology file created successfully!
		--running GROMPP to make a protein tpr..
		--GROMPP completed successfully!
		--Counting HIS residues in the PDB file..
		--Found HIS residues: 0
Files created successfully!


In [22]:
# get number of frames
try:
    with open("trj_check.out", "r") as file:
        number_of_frames = next(
            (line.split()[1] for line in file if line.startswith("Step")), None
        )
except FileNotFoundError:
    print(f"Error: File trj_check.out not found.")
    number_of_frames = None
print(number_of_frames)

151


In [23]:
conda_path = shutil.which("conda")
conda_activate_path = os.path.dirname(conda_path)

In [24]:
#conda_activate_path="/home/bio/ls/bin"
conda_gmxmmpbsa_name="gmxMMPBSA"

In [25]:
forcefield="amber99sb-ildn"
#FORCE_FIELD_PATH = "/home/bio/Desktop/jupyter_test/antibody_test/FORCE_FIELD"
mmpbsa_inFILE="mmpbsa_LinearPB_amber99SB_ILDN.in"
#MMPBSA_INFILE_PATH = "/home/bio/Desktop/jupyter_test/antibody_test/gmx_mmpbsa_in"
np_value = 32

In [26]:
# Example usage
gmx_mmpbsa(1, conda_activate_path, conda_gmxmmpbsa_name, cycle_number_MD_FOLDER, ConfName, RootName, forcefield, FORCE_FIELD_PATH, 
                 mmpbsa_inFILE, MMPBSA_INFILE_PATH , np_value, number_of_frames)

NP_value=32 	 number_of_frames=151 	 NP_used=16
Finished gmx_MMPBSA on cycle1


In [28]:
        #command = f"mpirun -np 16 gmx_MMPBSA MPI -O -i mmpbsa_LinearPB_amber99SB_ILDN.in -cs cycle1_newGRO.tpr -ci index.ndx " \
         # f"-cg 0 1 -ct cycle1_noPBC.xtc -cr ./cycle1_starting_protein.pdb -cp topol_protein.top " \
         # f"-eo gmx_MMPBSA_plot.csv -deo FINAL_DECOMP_MMPBSA.csv -nogui"

In [27]:
import csv

def extract_delta_energy_terms(input_filename, output_filename):
    try:
        with open(input_filename, 'r') as infile:
            lines = infile.readlines()  # 以行读取文件内容
    except FileNotFoundError:
        print(f"Error: {input_filename} file not found.")
        return

    # 查找 Delta Energy Terms 部分的开始和结束
    delta_start = False
    delta_lines = []

    for line in lines:
        # 查找 Delta Energy Terms 标题
        if "Delta Energy Terms" in line:
            delta_start = True
            continue  # 跳过该行
        # 当找到 Delta Energy Terms 部分后开始收集数据
        if delta_start:
            if line.strip() == "":  # 遇到空行结束
                break
            delta_lines.append(line.strip().split(','))  # 用逗号分割每行数据

    if not delta_lines:
        print("Error: 'Delta Energy Terms' data not found.")
        return

    # 将提取的数据写入新的 CSV 文件
    try:
        with open(output_filename, 'w', newline='') as outfile:
            writer = csv.writer(outfile)
            writer.writerows(delta_lines)  # 将提取的数据写入文件
        print(f"Delta Energy Terms extracted and saved to {output_filename}")
    except Exception as e:
        print(f"Error saving the file: {e}")



In [28]:
# 使用示例
input_filename = "gmx_MMPBSA_plot.csv"  # 输入的 CSV 文件
output_filename = "delta_energy_terms.csv"  # 输出的 CSV 文件
extract_delta_energy_terms(input_filename, output_filename)

Delta Energy Terms extracted and saved to delta_energy_terms.csv


In [29]:
# read the csv file
df = pd.read_csv('delta_energy_terms.csv')

# create df
output_data = []


for index, row in df.iterrows():
    # for each line
    
    # Frame #
    frame = row['Frame #']  
    
    # DeltaG(kJ/mol)
    delta_g = row['TOTAL']  
    
    # Coul(kJ/mol) = EEL + EEL14
    EEL = row['EEL']  
    EEL14 = row['1-4 EEL']  
    coul = EEL + EEL14
    
    # vdW(kJ/mol) = VDW + VDW14
    VDW = row['VDWAALS']  
    VDW14 = row['1-4 VDW']  
    vdW = VDW + VDW14
    
    # PolSol(kJ/mol) = EPB + ENPOLAR (if have)
    # for this infile the NpoSol is the value of [ENPOLAR], so it is included in the PolSol

    if pd.notna(row['EPB']) and pd.notna(row['ENPOLAR']):
        EPB = row['EPB']
        ENPOLAR = row['ENPOLAR']
        pol_sol = EPB + ENPOLAR
    else:
        pol_sol = row['EGB']  # if have
    
    # NpoSol(kJ/mol) = EDISPER or ESURF (if have)
    non_pol_solv = 0
    if pd.notna(row['EDISPER']):
        non_pol_solv = row['EDISPER']
    elif pd.notna(row['ESURF']):
        non_pol_solv = row['ESURF']
    
    # add the values 
    output_data.append({
        '# frame': frame,
        'DeltaG(kJ/mol)': delta_g,
        'Coul(kJ/mol)': coul,
        'vdW(kJ/mol)': vdW,
        'PolSol(kJ/mol)': pol_sol,
        'NpoSol(kJ/mol)': non_pol_solv
    })

# transfer
output_df = pd.DataFrame(output_data)

# write it in
output_df.to_csv('energy_plot_temp.csv', sep='\t', index=False)

In [30]:
import os
import shutil
import datetime
import time

# 配置参数
NUMframe = "all"  # 或者一个特定的数字

#logging.info("Cleaning files.")
# 处理 energy_plot_temp 文件
with open('./energy_plot_temp.csv', 'r') as f:
    lines = f.readlines()

# 获取表头
#header = lines[0]  # 第一行是表头

# 根据 NUMframe 判断截取行
if NUMframe == "all":
    selected_lines = lines[1:]  # 如果是 "all"，保留所有内容，包括表头
else:
    selected_lines = lines[int(NUMframe):]  # 保留表头，并从指定行开始截取

# 保存处理后的文件
with open('../energy_plot_temp.csv', 'w') as f:
    f.writelines(selected_lines)


# 清理文件
# 移动匹配的文件到指定目录
for filename in os.listdir('.'):
    if filename.startswith('_GMXMMPBSA'):
        shutil.move(filename, os.path.join(folders["REMOVED_FILES_FOLDER"], filename))
    elif '_temp' in filename:
        shutil.move(filename, os.path.join(folders["REMOVED_FILES_FOLDER"], filename))

# 删除备份和 .ff 文件
for filename in os.listdir('.'):
    if filename.endswith(('#', '~', '.ff')):
        os.remove(filename)

# 记录清理操作
with open(os.path.join(folders["REMOVED_FILES_FOLDER"], 'rm.out'), 'w') as rm_out_file:
    rm_out_file.write("Removed backup and .ff files.")

# 切换到指定目录
try:
    os.chdir(cycle_number_MD_FOLDER)
except FileNotFoundError:
    raise SystemExit(f"Cannot enter '{cycle_number_MD_FOLDER}' folder")



In [31]:
import numpy as np
import pandas as pd
import math

logging.info("Data Analysis.")
# 读取数据
input_file = 'energy_plot_temp.csv'  
# NEED TO CHANGE cycle${Cycle_Number}_results.dat
output_file = f'cycle1_results.dat'

# 假设文件数据为 tab-separated
df = pd.read_csv(input_file, delim_whitespace=True, header=None)

# 为了清晰，假设文件列对应以下字段
df.columns = ['frame', 'DeltaG', 'Coul', 'VdW', 'PolSol', 'NpoSol']

# 初始化变量
Population = len(df)
DeltaG = df['DeltaG'].sum()
Coul = df['Coul'].sum()
VdW = df['VdW'].sum()
PolSol = df['PolSol'].sum()
NpoSol = df['NpoSol'].sum()

# 计算 SF1 和 SF2
df['SF1'] = (df['Coul'] / 10) - (df['PolSol'] / 10) + (df['NpoSol'] * 10)
df['SF2'] = (3 * df['Coul']) + df['PolSol']

# 初始化变量
Canonical_AVG = 0.0
Canonical_AVG_w = 0.0

for _, row in df.iterrows():
    # 尝试将 DeltaG 转换为浮动类型
    DeltaG_temp = row[1]  # 使用列名访问 DeltaG
    # 计算指数加权值
    weight = math.exp(-int(DeltaG_temp / 2.479))
    # 累加计算
    Canonical_AVG += DeltaG_temp * weight
    Canonical_AVG_w += weight

# 归一化
if Canonical_AVG_w != 0:
    Canonical_AVG /= Canonical_AVG_w

# 平均值计算
mean_DeltaG = DeltaG / Population
mean_Coul = Coul / Population
mean_VdW = VdW / Population
mean_PolSol = PolSol / Population
mean_NpoSol = NpoSol / Population

mean_SF1 = mean_Coul / 10 - mean_PolSol / 10 + mean_NpoSol * 10
mean_SF2 = (3 * mean_Coul) + mean_PolSol


# 计算标准差
std_DeltaG = np.std(df['DeltaG'], ddof=1)
std_Coul = np.std(df['Coul'], ddof=1)
std_VdW = np.std(df['VdW'], ddof=1)
std_PolSol = np.std(df['PolSol'], ddof=1)
std_NpoSol = np.std(df['NpoSol'], ddof=1)
std_SF1 = std_Coul / 10 - std_PolSol / 10 + std_NpoSol * 10
std_SF2 = (3 * std_Coul) + std_PolSol

# 计算 DeltaG 在2sigma内的值
DeltaG_2s = df[(np.abs(df['DeltaG'] - mean_DeltaG) < 2 * std_DeltaG)]['DeltaG'].mean()

# 计算中位数
median_DeltaG = df['DeltaG'].median()
# 计算标准差
std_DeltaG = np.std(df['DeltaG'], ddof=1)  # ddof=1 表示使用样本标准差公式（贝塞尔校正）


# 创建警告信息列表
warnings = []

# 遍历每一行，检查是否超出 2 sigma
for i, row in df.iterrows():
    var = np.abs(row['DeltaG'] - mean_DeltaG)
    if var >= 2 * std_DeltaG:
        warnings.append(f"# WARNING: frame {row['frame']} is out of 2 sigma!!")

# 输出处理结果
with open(output_file, 'w') as f:
    # 写入头部信息
    f.write("# SF1=Coulomb/10-PolarSolvation/10+Non-PolarSolvation*10\n")
    f.write("# SF2=3*Coulomb+PolarSolvation\n")
    f.write("# C_AVG=norm(SUM Gi*e^BGi)\n")
    f.write(f"#frame\tDeltaG(kJ/mol)\tCoul(kJ/mol)\tVdW(kJ/mol)\tPolSol(kJ/mol)\tNpoSol(kJ/mol)\tSF1\tSF2\n")

    # 写入数据
    for _, row in df.iterrows():
        f.write(f"{row['frame']:<10}{row['DeltaG']:>12.3f}{row['Coul']:>13.3f}{row['VdW']:>13.3f}{row['PolSol']:>13.3f}{row['NpoSol']:>13.3f}{row['SF1']:>13.3f}{row['SF2']:>13.3f}\n")
    
     # 写入警告信息
    for warning in warnings:
        f.write(f"{warning}\n")   
   
    # 写入汇总信息表头
    f.write("\n# FINAL RESULTS\n")
    f.write(f"#frame\t{'DeltaG(kJ/mol)':>15}\t{'Coul(kJ/mol)':>15}\t{'VdW(kJ/mol)':>15}\t{'PolSol(kJ/mol)':>15}\t{'NpoSol(kJ/mol)':>15}\t{'SF1':>15}\t{'SF2':>15}\t{'Canonical_AVG':>15}\t{'MedianDeltaG(kJ/mol)':>15}\t{'DeltaG_2s(kJ/mol)':>15}\n")
    # 写入汇总结果
    f.write(f"#AVG\t{mean_DeltaG:>15.1f}\t{mean_Coul:>15.1f}\t{mean_VdW:>15.1f}\t{mean_PolSol:>15.1f}\t{mean_NpoSol:>15.1f}\t{mean_SF1:>15.1f}\t{mean_SF2:>15.1f}\t{Canonical_AVG:>15.1f}\t{median_DeltaG:>15.1f}\t{DeltaG_2s:>15.1f}\n")
    f.write(f"#STD\t{std_DeltaG:>15.1f}\t{std_Coul:>15.1f}\t{std_VdW:>15.1f}\t{std_PolSol:>15.1f}\t{std_NpoSol:>15.1f}\t{std_SF1:>15.1f}\t{std_SF2:>15.1f}\t{"nan":>15}\t{"nan":>15}\t{std_DeltaG:>15.1f}\n")

/tmp/ipykernel_10684/1971998278.py:11: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(input_file, delim_whitespace=True, header=None)
/tmp/ipykernel_10684/1971998278.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  DeltaG_temp = row[1]  # 使用列名访问 DeltaG


In [36]:
# write data in logging file 
cycle_number =1
# 打开文件并读取内容

try:
    with open(f'cycle{cycle_number}_results.dat', 'r') as file:
        lines = file.readlines()
except FileNotFoundError:
    logging.error(f"File not found: cycle{cycle_number}_results.dat")
    exit()

# 初始化变量存储表头、平均值和标准差
header = None
avg_values = None
std_values = None

# 逐行解析文件内容
for line in lines:
    if line.startswith("#frame"):
        header = line.strip().lstrip("#").split()
    elif line.startswith("#AVG"):
        avg_values = line.strip().lstrip("#").split()
    elif line.startswith("#STD"):
        std_values = line.strip().lstrip("#").split()

# 验证数据完整性
if not header or not avg_values or not std_values:
    logging.error("Missing required data (header, AVG, or STD).")
    exit()

# 格式化日志输出
log_message = (
    f"Results for cycle1:\n"
    f"Headers: {' | '.join(header)}\n"
    f"AVG Values: {' | '.join(avg_values)}\n"
    f"STD Values: {' | '.join(std_values)}"
)

# 输出合并后的日志信息
logging.info(log_message)


# 同时打印到控制台
print("Headers:", " | ".join(header))
print("AVG Values:", " | ".join(avg_values))
print("STD Values:", " | ".join(std_values))

Headers: frame | DeltaG(kJ/mol) | Coul(kJ/mol) | VdW(kJ/mol) | PolSol(kJ/mol) | NpoSol(kJ/mol) | SF1 | SF2 | Canonical_AVG | MedianDeltaG(kJ/mol) | DeltaG_2s(kJ/mol)
AVG Values: AVG | -72.4 | -264.3 | -97.6 | 289.5 | 0.0 | -55.4 | -503.4 | -94.1 | -72.8 | -72.4
STD Values: STD | 8.6 | 21.8 | 5.3 | 21.0 | 0.0 | 0.1 | 86.5 | nan | nan | 8.6


In [33]:
shutil.move(output_file, os.path.join(folders["results"], output_file))

'/home/bio/Desktop/jupyter_test/antibody_test/output_20241203_123324/configuration/RESULTS/cycle1_results.dat'

In [34]:
import os

def append_results(cycle_number, results_folder):
    # 定义输出文件路径
    output_file = os.path.join(results_folder, "MoleculesResults.dat")
    # 定义输入文件路径
    input_file = os.path.join(results_folder, f"cycle{cycle_number}_results.dat")
    
    # 打开输出文件，追加写入模式
    with open(output_file, "a") as outfile:
        # 写入 Cycle_Number，左对齐，占10个字符宽度
        outfile.write(f"{cycle_number:<5}\t")
        
        # 从输入文件中提取包含 "#AVG" 的行并处理
        if os.path.exists(input_file):
            with open(input_file, "r") as infile:
                for line in infile:
                    if "#AVG" in line:
                        # 提取从第二列开始的内容（假设以制表符分隔）
                        columns = line.strip().split("\t")[1:]
                        outfile.write("\t".join(columns) + "\n")

In [35]:
append_results(1,folders['results'])

In [39]:
import os
import shutil
import logging
from datetime import datetime


# 清理文件 NO SUCH FILE NEED TO IMPROVE
def clean_up(removed_files_folder):
    logging.info("Cleaning files.")
    try:
        for pattern in ["*#*", "*~*", "*temp*"]:
            for filename in os.listdir('.'):
                if filename.startswith(pattern):
                    file_path = os.path.join('.', filename)
                    if os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                    else:
                        os.remove(file_path)
        # 记录清理操作的日志
        with open(os.path.join(removed_files_folder, 'rm.out'), 'w') as f:
            f.write("Cleanup completed.")
    except Exception as e:
        logging.error(f"Error during cleanup: {e}")

In [40]:
clean_up(folders["REMOVED_FILES_FOLDER"])

In [45]:
# 移动和复制文件
def move_and_copy_files(cycle_number, repository_folder, cycle_number_MD_folder, root_name):
    logging.info("Moving files.")
    try:
        # 移动文件
        for filename in os.listdir('.'):
            if filename.startswith(f'RUN1_cycle{cycle_number}_prot') or filename.endswith('.out') or filename == root_name:
                shutil.move(filename, os.path.join(repository_folder, filename))
        
        # 复制文件
        shutil.copy('system_Compl_MDstart.gro', os.path.join(repository_folder, f'system_cycle{cycle_number}_MDstart.gro'))
        if os.path.isdir(cycle_number_MD_folder):
            shutil.copytree(cycle_number_MD_folder, os.path.join(repository_folder, f'cycle{cycle_number}_MD'))
        
    except Exception as e:
        logging.error(f"Error during file operations: {e}")

In [46]:
move_and_copy_files(1, folders["repository"], cycle_number_MD_FOLDER, RootName)

In [47]:
import os
import shutil

# 假设这些变量已定义
removed_files_folder = folders["REMOVED_FILES_FOLDER"] # 被删除文件的目标文件夹
current_conf_path = configuration_path  # 当前配置路径
cycle_number = 1  # 示例初始周期号

# 移动文件到 removed_files_folder
for filename in os.listdir('.'):
    if os.path.isfile(filename):  # 检查是否是普通文件
        shutil.move(filename, os.path.join(removed_files_folder, filename))

# 删除当前目录下的所有文件
for filename in os.listdir('.'):
    file_path = os.path.join('.', filename)
    if os.path.isfile(file_path):
        os.remove(file_path)

# 删除特定的目录
cycle_number_md_folder = os.path.join(current_conf_path, f"cycle{cycle_number}_MD")
if os.path.exists(cycle_number_md_folder):
    shutil.rmtree(cycle_number_md_folder)

# 更新周期号及相关变量
cycle_number += 1
conf_name = f"cycle{cycle_number}"
root_name = f"cycle{cycle_number}_BE"
#cycle_number_md_folder = os.path.join(current_conf_path, f"cycle{cycle_number}_MD")

# 输出相关信息
print(f"Cycle Number: {cycle_number}")
print(f"Configuration Name: {conf_name}")
print(f"Root Name: {root_name}")
#print(f"MD Folder Path: {cycle_number_md_folder}")

Cycle Number: 2
Configuration Name: cycle2
Root Name: cycle2_BE
